In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
# Load data
train = pd.read_csv('/home/ayushz/Kaggle/Datasets/train.csv')
test = pd.read_csv('/home/ayushz/Kaggle/Datasets/test.csv')

# Initial data inspection
train.head()
# train_data.info()
# train_data.describe()

# Replace 0s in dimensions with NaN and attempt to fill (Note: fillna here doesn't work in-place)
for col in ['x','y','z']:
    train.loc[train[col] <= 0, col] = np.nan
    test.loc[test[col] <= 0, col] = np.nan
    train[col].fillna(train[col].median())
    test[col].fillna(train[col].median())

# Initialize LabelEncoder
le = LabelEncoder()

# Preprocess and encode 'train' data
train['cut'] = train['cut'].str.strip()
train['cut'] = le.fit_transform(train['cut'])
train['color'] = le.fit_transform(train['color'])
train['clarity'] = le.fit_transform(train['clarity'])

# Preprocess and encode 'test' data
test['cut'] = test['cut'].str.strip()
test['cut'] = le.fit_transform(test['cut'])
test['color'] = le.fit_transform(test['color'])
test['clarity'] = le.fit_transform(test['clarity'])

# Clean x, y, z (dimensions) by replacing non-positive values with the median
for col in ['x','y','z']:
    train[col] = train[col].apply(lambda v: v if v > 0 else train[col].median())
    test[col]  = test[col].apply(lambda v: v if v > 0 else train[col].median())

# Feature engineering
train['volume'] = train['x'] * train['y'] * train['z']
test['volume']  = test['x'] * test['y'] * test['z']

train['carat_per_volume'] = train['carat'] / (train['volume'] + 1e-9)
test['carat_per_volume']  = test['carat'] / (test['volume'] + 1e-9)

# Prepare data for modeling
X = train.drop(['price','id'],axis=1)
y = train['price']

X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define model parameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'subsample': 0.9,
    'reg_lambda': 0.0,
    'reg_alpha': 0.2222,
    'boosting_type': 'gbdt',
    'num_leaves': 65,
    'min_split_gain': 0.2222,
    'min_child_samples': 55,
    'max_depth': 3,
    'max_bin': 250,
    'learning_rate': 0.1,
    'colsample_bytree': 0.7,
    'verbose': -1
}

# Train the model with early stopping
model_lgb = lgb.train(
    params,
    train_data,
    valid_sets=[test_data],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)

# Predict on the test set using the best iteration
y_pred = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)
mae_lgb = mean_absolute_error(y_test, y_pred)
print(f"LightGBM MAE: {mae_lgb}")



# --- Submission Generation ---

# 1. Prepare the competition test data for prediction
# Ensure it has the same columns in the same order as the training data
X_submission = test[X_train.columns]

# 2. Predict on the competition test set
# Use the model trained earlier ('model_lgb')
competition_predictions = model_lgb.predict(X_submission, num_iteration=model_lgb.best_iteration)

# 3. Create the submission file in the correct format
submission_df = pd.DataFrame({
    'id': test['id'],
    'price': competition_predictions
})

# Optional: Ensure prices are not negative
submission_df['price'] = submission_df['price'].clip(0)

# # 4. Save the submission file
# submission_df.to_csv('submission.csv', index=False)

# print("Submission file 'submission.csv' created successfully!")
# print(submission_df.head())

# # --- Add this after your model is trained ---
# import joblib

# # Save the model to a file
# joblib.dump(model_lgb, 'lgbm_model.joblib')

# print("Model saved successfully!")

/home/ayushz/.local/lib/python3.12/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 2376.76
Early stopping, best iteration is:
[73]	valid_0's rmse: 2374.94
LightGBM MAE: 1373.207890510665


In [2]:
from sklearn.metrics import r2_score, mean_absolute_error

# Predict on the test set (already done)
y_pred = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)

# Calculate R² score
r2_lgb = r2_score(y_test, y_pred)
print(f"LightGBM R² Score: {r2_lgb}")

# Optional: Already calculating MAE
mae_lgb = mean_absolute_error(y_test, y_pred)
print(f"LightGBM MAE: {mae_lgb}")




LightGBM R² Score: 0.6535287243179684
LightGBM MAE: 1373.207890510665


In [10]:
# Advanced Feature Engineering and Model Optimization for R² >= 0.68
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler, QuantileTransformer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')

print("Starting advanced optimization to achieve R² ≥ 0.68...")

# Load fresh data
train_fresh = pd.read_csv('/home/ayushz/Kaggle/Datasets/train.csv')
test_fresh = pd.read_csv('/home/ayushz/Kaggle/Datasets/test.csv')

print(f"Train shape: {train_fresh.shape}, Test shape: {test_fresh.shape}")

# Advanced outlier detection using IQR method
def remove_outliers_iqr(df, column, factor=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers more aggressively
original_len = len(train_fresh)
train_fresh = remove_outliers_iqr(train_fresh, 'price', factor=1.2)
print(f"Removed {original_len - len(train_fresh)} price outliers")

# Remove dimension outliers
for dim in ['x', 'y', 'z']:
    train_fresh = remove_outliers_iqr(train_fresh, dim, factor=2.0)

print(f"Final training data shape: {train_fresh.shape}")

# More sophisticated missing value handling
for col in ['x','y','z']:
    # Replace very small and zero values
    train_fresh.loc[train_fresh[col] <= 0.01, col] = np.nan
    test_fresh.loc[test_fresh[col] <= 0.01, col] = np.nan
    
    # Group-based imputation by cut and clarity
    for cut_val in train_fresh['cut'].unique():
        for clarity_val in train_fresh['clarity'].unique():
            mask_train = (train_fresh['cut'] == cut_val) & (train_fresh['clarity'] == clarity_val)
            mask_test = (test_fresh['cut'] == cut_val) & (test_fresh['clarity'] == clarity_val)
            
            if mask_train.sum() > 0 and train_fresh.loc[mask_train, col].notna().sum() > 0:
                median_val = train_fresh.loc[mask_train, col].median()
                train_fresh.loc[mask_train & train_fresh[col].isna(), col] = median_val
                test_fresh.loc[mask_test & test_fresh[col].isna(), col] = median_val
    
    # Fill remaining with overall median
    overall_median = train_fresh[col].median()
    train_fresh[col].fillna(overall_median, inplace=True)
    test_fresh[col].fillna(overall_median, inplace=True)

# Clean categorical variables
train_fresh['cut'] = train_fresh['cut'].str.strip()
test_fresh['cut'] = test_fresh['cut'].str.strip()

print("Advanced data cleaning completed!")

Starting advanced optimization to achieve R² ≥ 0.68...
Train shape: (20000, 11), Test shape: (30000, 10)
Removed 1758 price outliers
Final training data shape: (18239, 11)
Advanced data cleaning completed!
Advanced data cleaning completed!


In [11]:
# Advanced Feature Engineering with Domain Knowledge
print("Creating advanced features...")

# Consistent encoding
le_cut = LabelEncoder()
le_color = LabelEncoder()
le_clarity = LabelEncoder()

# Fit on combined data
all_data = pd.concat([
    train_fresh[['cut', 'color', 'clarity']], 
    test_fresh[['cut', 'color', 'clarity']]
])

le_cut.fit(all_data['cut'])
le_color.fit(all_data['color'])
le_clarity.fit(all_data['clarity'])

train_fresh['cut_encoded'] = le_cut.transform(train_fresh['cut'])
train_fresh['color_encoded'] = le_color.transform(train_fresh['color'])
train_fresh['clarity_encoded'] = le_clarity.transform(train_fresh['clarity'])

test_fresh['cut_encoded'] = le_cut.transform(test_fresh['cut'])
test_fresh['color_encoded'] = le_color.transform(test_fresh['color'])
test_fresh['clarity_encoded'] = le_clarity.transform(test_fresh['clarity'])

# Basic geometric features
train_fresh['volume'] = train_fresh['x'] * train_fresh['y'] * train_fresh['z']
test_fresh['volume'] = test_fresh['x'] * test_fresh['y'] * test_fresh['z']

train_fresh['surface_area'] = 2 * (train_fresh['x']*train_fresh['y'] + 
                                  train_fresh['x']*train_fresh['z'] + 
                                  train_fresh['y']*train_fresh['z'])
test_fresh['surface_area'] = 2 * (test_fresh['x']*test_fresh['y'] + 
                                 test_fresh['x']*test_fresh['z'] + 
                                 test_fresh['y']*test_fresh['z'])

# Advanced ratios and transformations
train_fresh['carat_per_volume'] = train_fresh['carat'] / (train_fresh['volume'] + 1e-8)
test_fresh['carat_per_volume'] = test_fresh['carat'] / (test_fresh['volume'] + 1e-8)

train_fresh['density'] = train_fresh['carat'] / (train_fresh['volume'] + 1e-8)
test_fresh['density'] = test_fresh['carat'] / (test_fresh['volume'] + 1e-8)

# Aspect ratios
train_fresh['aspect_xy'] = train_fresh['x'] / (train_fresh['y'] + 1e-8)
test_fresh['aspect_xy'] = test_fresh['x'] / (test_fresh['y'] + 1e-8)

train_fresh['aspect_xz'] = train_fresh['x'] / (train_fresh['z'] + 1e-8)
test_fresh['aspect_xz'] = test_fresh['x'] / (test_fresh['z'] + 1e-8)

# Diamond quality score (domain knowledge)
# Better cut, color, clarity = higher value
cut_quality = {'Fair': 1, 'Good': 2, 'Very Good': 3, 'Premium': 4, 'Ideal': 5}
color_quality = {'J': 1, 'I': 2, 'H': 3, 'G': 4, 'F': 5, 'E': 6, 'D': 7}
clarity_quality = {'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2': 4, 'VS1': 5, 'VVS2': 6, 'VVS1': 7, 'IF': 8}

train_fresh['cut_quality'] = train_fresh['cut'].map(cut_quality).fillna(3)
test_fresh['cut_quality'] = test_fresh['cut'].map(cut_quality).fillna(3)

train_fresh['color_quality'] = train_fresh['color'].map(color_quality).fillna(4)
test_fresh['color_quality'] = test_fresh['color'].map(color_quality).fillna(4)

train_fresh['clarity_quality'] = train_fresh['clarity'].map(clarity_quality).fillna(4)
test_fresh['clarity_quality'] = test_fresh['clarity'].map(clarity_quality).fillna(4)

train_fresh['overall_quality'] = (train_fresh['cut_quality'] + 
                                 train_fresh['color_quality'] + 
                                 train_fresh['clarity_quality'])
test_fresh['overall_quality'] = (test_fresh['cut_quality'] + 
                                test_fresh['color_quality'] + 
                                test_fresh['clarity_quality'])

# Log transformations for skewed features
train_fresh['log_carat'] = np.log1p(train_fresh['carat'])
test_fresh['log_carat'] = np.log1p(test_fresh['carat'])

train_fresh['log_volume'] = np.log1p(train_fresh['volume'])
test_fresh['log_volume'] = np.log1p(test_fresh['volume'])

# Polynomial features for key variables
for col in ['carat', 'volume', 'overall_quality']:
    train_fresh[f'{col}_squared'] = train_fresh[col] ** 2
    test_fresh[f'{col}_squared'] = test_fresh[col] ** 2
    
    train_fresh[f'{col}_cubed'] = train_fresh[col] ** 3
    test_fresh[f'{col}_cubed'] = test_fresh[col] ** 3

# Interaction features
train_fresh['carat_quality'] = train_fresh['carat'] * train_fresh['overall_quality']
test_fresh['carat_quality'] = test_fresh['carat'] * test_fresh['overall_quality']

train_fresh['volume_quality'] = train_fresh['volume'] * train_fresh['overall_quality']
test_fresh['volume_quality'] = test_fresh['volume'] * test_fresh['overall_quality']

# Binning features
train_fresh['carat_bin'] = pd.cut(train_fresh['carat'], bins=10, labels=False)
test_fresh['carat_bin'] = pd.cut(test_fresh['carat'], bins=10, labels=False)

print("Advanced feature engineering completed!")
print(f"Train features: {train_fresh.shape[1]}, Test features: {test_fresh.shape[1]}")

Creating advanced features...
Advanced feature engineering completed!
Train features: 35, Test features: 34


In [13]:
# Optimized LightGBM and CatBoost for R² >= 0.68
import catboost as cb
from catboost import CatBoostRegressor
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

print("Training optimized LightGBM and CatBoost models...")

# Select features
feature_cols = [
    'carat', 'cut_encoded', 'color_encoded', 'clarity_encoded', 'depth', 'table',
    'x', 'y', 'z', 'volume', 'surface_area', 'carat_per_volume', 'density',
    'aspect_xy', 'aspect_xz', 'cut_quality', 'color_quality', 'clarity_quality',
    'overall_quality', 'log_carat', 'log_volume', 'carat_squared', 'volume_squared',
    'overall_quality_squared', 'carat_cubed', 'volume_cubed', 'overall_quality_cubed',
    'carat_quality', 'volume_quality', 'carat_bin'
]

X_advanced = train_fresh[feature_cols]
y_advanced = train_fresh['price']

print(f"Feature matrix shape: {X_advanced.shape}")
print(f"Target shape: {y_advanced.shape}")

# Advanced train-test split with stratification
X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(
    X_advanced, y_advanced, test_size=0.2, random_state=42, 
    stratify=pd.cut(y_advanced, bins=10, labels=False)
)

print(f"Advanced train set: {X_train_adv.shape}")
print(f"Advanced test set: {X_test_adv.shape}")

# Dictionary to store best models
best_models = {}

# 1. Ultra-optimized LightGBM with aggressive hyperparameters
print("Training ultra-optimized LightGBM...")

params_lgb_ultra = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 300,          # Increased significantly
    'max_depth': 15,            # Increased depth
    'learning_rate': 0.01,      # Lower learning rate for more iterations
    'feature_fraction': 0.95,   # Use more features
    'bagging_fraction': 0.95,   # Use more data
    'bagging_freq': 5,
    'min_child_samples': 5,     # Lower minimum samples
    'min_split_gain': 0.01,     # Lower split gain threshold
    'reg_alpha': 0.01,          # Light regularization
    'reg_lambda': 0.01,
    'subsample': 0.95,
    'colsample_bytree': 0.95,
    'max_bin': 500,             # More bins for better splits
    'verbose': -1,
    'random_state': 42,
    'extra_trees': True,        # Extra randomization
    'force_row_wise': True,
    'device': 'cpu',
    'num_threads': -1
}

train_data_lgb = lgb.Dataset(X_train_adv, label=y_train_adv)
valid_data_lgb = lgb.Dataset(X_test_adv, label=y_test_adv, reference=train_data_lgb)

model_lgb_ultra = lgb.train(
    params_lgb_ultra,
    train_data_lgb,
    valid_sets=[valid_data_lgb],
    num_boost_round=8000,       # Much higher iterations
    callbacks=[
        lgb.early_stopping(300),  # Higher patience
        lgb.log_evaluation(0)     # Silent
    ]
)

y_pred_lgb = model_lgb_ultra.predict(X_test_adv, num_iteration=model_lgb_ultra.best_iteration)
r2_lgb = r2_score(y_test_adv, y_pred_lgb)
mae_lgb = mean_absolute_error(y_test_adv, y_pred_lgb)

best_models['Ultra_LightGBM'] = {
    'model': model_lgb_ultra,
    'r2': r2_lgb,
    'mae': mae_lgb,
    'predictions': y_pred_lgb
}

print(f"Ultra LightGBM - R²: {r2_lgb:.4f}, MAE: {mae_lgb:.2f}")

# 2. Optimized CatBoost
print("Training optimized CatBoost...")

# CatBoost parameters optimized for diamond price prediction
catboost_params = {
    'iterations': 5000,
    'learning_rate': 0.01,
    'depth': 12,
    'l2_leaf_reg': 3,
    'border_count': 254,
    'feature_border_type': 'GreedyLogSum',
    'bagging_temperature': 0.2,
    'random_strength': 0.2,
    'od_type': 'Iter',
    'od_wait': 300,
    'random_seed': 42,
    'verbose': False,
    'thread_count': -1,
    'task_type': 'CPU',
    'bootstrap_type': 'Bernoulli',  # Changed from Bayesian
    'subsample': 0.95,
    'sampling_frequency': 'PerTreeLevel',
    'leaf_estimation_method': 'Newton',
    'grow_policy': 'SymmetricTree',
    'penalties_coefficient': 1,
    'boosting_type': 'Plain',
    'model_shrink_rate': 0.1,
    'model_shrink_mode': 'Constant',
    'langevin': False,
    'diffusion_temperature': 10000,
    'posterior_sampling': False,
    'boost_from_average': True
}

model_catboost = CatBoostRegressor(**catboost_params)
model_catboost.fit(
    X_train_adv, y_train_adv,
    eval_set=(X_test_adv, y_test_adv),
    verbose=False,
    plot=False
)

y_pred_cb = model_catboost.predict(X_test_adv)
r2_cb = r2_score(y_test_adv, y_pred_cb)
mae_cb = mean_absolute_error(y_test_adv, y_pred_cb)

best_models['Optimized_CatBoost'] = {
    'model': model_catboost,
    'r2': r2_cb,
    'mae': mae_cb,
    'predictions': y_pred_cb
}

print(f"Optimized CatBoost - R²: {r2_cb:.4f}, MAE: {mae_cb:.2f}")

# 3. Alternative LightGBM with different approach
print("Training alternative LightGBM with DART boosting...")

params_lgb_dart = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'dart',    # DART boosting
    'num_leaves': 255,
    'max_depth': 12,
    'learning_rate': 0.02,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'min_child_samples': 10,
    'min_split_gain': 0.02,
    'reg_alpha': 0.05,
    'reg_lambda': 0.05,
    'drop_rate': 0.1,           # DART specific
    'max_drop': 50,             # DART specific
    'skip_drop': 0.5,           # DART specific
    'xgboost_dart_mode': False,
    'uniform_drop': False,
    'verbose': -1,
    'random_state': 42,
    'force_row_wise': True
}

model_lgb_dart = lgb.train(
    params_lgb_dart,
    train_data_lgb,
    valid_sets=[valid_data_lgb],
    num_boost_round=4000,
    callbacks=[
        lgb.early_stopping(200),
        lgb.log_evaluation(0)
    ]
)

y_pred_dart = model_lgb_dart.predict(X_test_adv, num_iteration=model_lgb_dart.best_iteration)
r2_dart = r2_score(y_test_adv, y_pred_dart)
mae_dart = mean_absolute_error(y_test_adv, y_pred_dart)

best_models['LightGBM_DART'] = {
    'model': model_lgb_dart,
    'r2': r2_dart,
    'mae': mae_dart,
    'predictions': y_pred_dart
}

print(f"LightGBM DART - R²: {r2_dart:.4f}, MAE: {mae_dart:.2f}")

print("\\nAll specialized models completed!")

Training optimized LightGBM and CatBoost models...
Feature matrix shape: (18239, 30)
Target shape: (18239,)
Advanced train set: (14591, 30)
Advanced test set: (3648, 30)
Training ultra-optimized LightGBM...
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[362]	valid_0's rmse: 1418.22
Ultra LightGBM - R²: 0.6780, MAE: 921.25
Training optimized CatBoost...


CatBoostError: catboost/private/libs/options/bootstrap_options.cpp:16: Error: bayesian bootstrap doesn't support 'subsample' option

In [14]:
# Continue with CatBoost (fixed) and additional LightGBM optimization
print("\\nContinuing with fixed CatBoost and more LightGBM variants...")

# 2. Fixed CatBoost
print("Training fixed CatBoost...")
try:
    catboost_params_fixed = {
        'iterations': 4000,
        'learning_rate': 0.015,
        'depth': 10,
        'l2_leaf_reg': 5,
        'border_count': 200,
        'bagging_temperature': 0.3,
        'random_strength': 0.3,
        'od_type': 'Iter',
        'od_wait': 200,
        'random_seed': 42,
        'verbose': False,
        'thread_count': -1,
        'bootstrap_type': 'Bernoulli',
        'subsample': 0.9,
        'leaf_estimation_method': 'Newton',
        'grow_policy': 'SymmetricTree',
        'boost_from_average': True,
        'auto_class_weights': None
    }
    
    model_catboost_fixed = CatBoostRegressor(**catboost_params_fixed)
    model_catboost_fixed.fit(
        X_train_adv, y_train_adv,
        eval_set=(X_test_adv, y_test_adv),
        verbose=False,
        plot=False,
        early_stopping_rounds=200
    )
    
    y_pred_cb_fixed = model_catboost_fixed.predict(X_test_adv)
    r2_cb_fixed = r2_score(y_test_adv, y_pred_cb_fixed)
    mae_cb_fixed = mean_absolute_error(y_test_adv, y_pred_cb_fixed)
    
    best_models['Fixed_CatBoost'] = {
        'model': model_catboost_fixed,
        'r2': r2_cb_fixed,
        'mae': mae_cb_fixed,
        'predictions': y_pred_cb_fixed
    }
    
    print(f"Fixed CatBoost - R²: {r2_cb_fixed:.4f}, MAE: {mae_cb_fixed:.2f}")
    
except Exception as e:
    print(f"CatBoost failed: {e}")
    print("Continuing without CatBoost...")

# 4. Ultra-fine-tuned LightGBM to push R² above 0.68
print("Training ultra-fine-tuned LightGBM to exceed 0.68...")

params_lgb_ultra_fine = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 350,          # Even higher
    'max_depth': 18,            # Deeper trees
    'learning_rate': 0.005,     # Very low learning rate
    'feature_fraction': 0.98,   # Use almost all features
    'bagging_fraction': 0.98,   # Use almost all data
    'bagging_freq': 3,
    'min_child_samples': 3,     # Very low minimum samples
    'min_split_gain': 0.005,    # Very low split gain
    'reg_alpha': 0.005,         # Minimal regularization
    'reg_lambda': 0.005,
    'subsample': 0.98,
    'colsample_bytree': 0.98,
    'max_bin': 600,             # Even more bins
    'verbose': -1,
    'random_state': 42,
    'extra_trees': True,
    'force_row_wise': True,
    'device': 'cpu',
    'num_threads': -1,
    'min_child_weight': 0.001,
    'min_data_in_leaf': 3,
    'lambda_l1': 0.005,
    'lambda_l2': 0.005
}

model_lgb_ultra_fine = lgb.train(
    params_lgb_ultra_fine,
    train_data_lgb,
    valid_sets=[valid_data_lgb],
    num_boost_round=12000,      # Very high iterations
    callbacks=[
        lgb.early_stopping(500), # Very high patience
        lgb.log_evaluation(0)
    ]
)

y_pred_ultra_fine = model_lgb_ultra_fine.predict(X_test_adv, num_iteration=model_lgb_ultra_fine.best_iteration)
r2_ultra_fine = r2_score(y_test_adv, y_pred_ultra_fine)
mae_ultra_fine = mean_absolute_error(y_test_adv, y_pred_ultra_fine)

best_models['Ultra_Fine_LightGBM'] = {
    'model': model_lgb_ultra_fine,
    'r2': r2_ultra_fine,
    'mae': mae_ultra_fine,
    'predictions': y_pred_ultra_fine
}

print(f"Ultra-Fine LightGBM - R²: {r2_ultra_fine:.4f}, MAE: {mae_ultra_fine:.2f}")

print("\\nAll optimized models completed!")

\nContinuing with fixed CatBoost and more LightGBM variants...
Training fixed CatBoost...
CatBoost failed: CatBoostRegressor.__init__() got an unexpected keyword argument 'auto_class_weights'
Continuing without CatBoost...
Training ultra-fine-tuned LightGBM to exceed 0.68...


[LightGBM] [Fatal] Cannot change max_bin after constructed Dataset handle.


LightGBMError: Cannot change max_bin after constructed Dataset handle.

In [15]:
# Quick ensemble and final results with current best models
print("Creating final results with achieved models...")

print("\\n" + "="*60)
print("FINAL MODEL PERFORMANCE SUMMARY")
print("="*60)

# Show current best results
for model_name, info in best_models.items():
    print(f"{model_name:20} - R²: {info['r2']:.4f}, MAE: {info['mae']:.2f}")

# Find the single best model
best_single_model = max(best_models.items(), key=lambda x: x[1]['r2'])
best_model_name = best_single_model[0]
best_r2_single = best_single_model[1]['r2']

print(f"\\nBest Single Model: {best_model_name} (R² = {best_r2_single:.4f})")

# Since we have multiple good LightGBM models, let's ensemble them
lightgbm_models = {k: v for k, v in best_models.items() if 'LightGBM' in k}

if len(lightgbm_models) > 1:
    print("\\nCreating LightGBM ensemble...")
    
    # Weight by R² performance
    lgb_r2_scores = [info['r2'] for info in lightgbm_models.values()]
    lgb_weights = np.array(lgb_r2_scores) / np.sum(lgb_r2_scores)
    
    print("LightGBM ensemble weights:")
    for i, (name, _) in enumerate(lightgbm_models.items()):
        print(f"  {name}: {lgb_weights[i]:.3f}")
    
    # Create ensemble predictions
    lgb_predictions_list = [info['predictions'] for info in lightgbm_models.values()]
    y_pred_lgb_ensemble = np.average(lgb_predictions_list, axis=0, weights=lgb_weights)
    
    r2_lgb_ensemble = r2_score(y_test_adv, y_pred_lgb_ensemble)
    mae_lgb_ensemble = mean_absolute_error(y_test_adv, y_pred_lgb_ensemble)
    
    print(f"\\nLightGBM Ensemble - R²: {r2_lgb_ensemble:.4f}, MAE: {mae_lgb_ensemble:.2f}")
    
    # Check if ensemble improves performance
    if r2_lgb_ensemble > best_r2_single:
        final_r2 = r2_lgb_ensemble
        final_approach = "LightGBM_Ensemble"
        use_ensemble = True
        print(f"✅ Ensemble improves performance!")
    else:
        final_r2 = best_r2_single
        final_approach = best_model_name
        use_ensemble = False
        print(f"Best single model performs better.")
else:
    final_r2 = best_r2_single
    final_approach = best_model_name
    use_ensemble = False

print("\\n" + "="*60)
print("🎯 FINAL RESULTS")
print("="*60)
print(f"Best Approach: {final_approach}")
print(f"Final R² Score: {final_r2:.4f}")
print(f"Target (≥ 0.68): {'✅ ACHIEVED!' if final_r2 >= 0.68 else f'❌ CLOSE! Need +{(0.68 - final_r2):.4f}'}")
print(f"Improvement from baseline: +{(final_r2 - 0.6535):.4f}")
print("="*60)

Creating final results with achieved models...
\n============================================================
FINAL MODEL PERFORMANCE SUMMARY
Ultra_LightGBM       - R²: 0.6780, MAE: 921.25
\nBest Single Model: Ultra_LightGBM (R² = 0.6780)
\n============================================================
🎯 FINAL RESULTS
Best Approach: Ultra_LightGBM
Final R² Score: 0.6780
Target (≥ 0.68): ❌ CLOSE! Need +0.0020
Improvement from baseline: +0.0245


In [16]:
# Generate final submission with best approach
print("\\nGenerating final submission...")

# Prepare test data
X_test_submission = test_fresh[feature_cols]
print(f"Test submission shape: {X_test_submission.shape}")

# Generate predictions based on best approach
if use_ensemble and len(lightgbm_models) > 1:
    print("Using LightGBM ensemble for final predictions...")
    final_preds = []
    for name, info in lightgbm_models.items():
        model = info['model']
        pred = model.predict(X_test_submission, num_iteration=model.best_iteration)
        final_preds.append(pred)
    
    submission_predictions = np.average(final_preds, axis=0, weights=lgb_weights)
    model_used = "LightGBM_Ensemble"
else:
    print(f"Using best single model: {best_model_name}")
    best_model = best_models[best_model_name]['model']
    submission_predictions = best_model.predict(X_test_submission, num_iteration=best_model.best_iteration)
    model_used = best_model_name

# Create final submission DataFrame
final_submission = pd.DataFrame({
    'id': test_fresh['id'],
    'price': submission_predictions
})

# Ensure positive prices and apply reasonable bounds
final_submission['price'] = final_submission['price'].clip(lower=0)

# Save the submission
filename = 'optimized_lgb_submission.csv'
final_submission.to_csv(filename, index=False)

print(f"\\n📊 SUBMISSION DETAILS")
print(f"File saved as: {filename}")
print(f"Model used: {model_used}")
print(f"R² Score achieved: {final_r2:.4f}")
print(f"Predictions shape: {final_submission.shape}")

print(f"\\n📈 PRICE STATISTICS")
print(f"Min price: ${final_submission['price'].min():.2f}")
print(f"Max price: ${final_submission['price'].max():.2f}")
print(f"Mean price: ${final_submission['price'].mean():.2f}")
print(f"Median price: ${final_submission['price'].median():.2f}")
print(f"Std deviation: ${final_submission['price'].std():.2f}")

# Show sample predictions
print(f"\\n🔍 SAMPLE PREDICTIONS")
print(final_submission.head(10))

# Save the best model(s)
import joblib

print(f"\\n💾 SAVING MODELS")
if use_ensemble and len(lightgbm_models) > 1:
    for i, (name, info) in enumerate(lightgbm_models.items()):
        safe_name = name.lower().replace('_', '-').replace(' ', '-')
        filename_model = f'{safe_name}-r2-{info["r2"]:.4f}.joblib'
        joblib.dump(info['model'], filename_model)
        print(f"  Saved: {filename_model}")
else:
    safe_name = best_model_name.lower().replace('_', '-').replace(' ', '-')
    filename_model = f'{safe_name}-r2-{final_r2:.4f}.joblib'
    joblib.dump(best_models[best_model_name]['model'], filename_model)
    print(f"  Saved: {filename_model}")

print("\\n🏆 OPTIMIZATION COMPLETE!")
print(f"Achieved R² = {final_r2:.4f}")
print(f"Target was R² ≥ 0.68")
if final_r2 >= 0.68:
    print("🎉 TARGET ACHIEVED!")
else:
    print(f"🎯 Very close! Only {(0.68 - final_r2):.4f} away from target.")

\nGenerating final submission...
Test submission shape: (30000, 30)
Using best single model: Ultra_LightGBM
\n📊 SUBMISSION DETAILS
File saved as: optimized_lgb_submission.csv
Model used: Ultra_LightGBM
R² Score achieved: 0.6780
Predictions shape: (30000, 2)
\n📈 PRICE STATISTICS
Min price: $775.81
Max price: $7900.52
Mean price: $3143.83
Median price: $2512.74
Std deviation: $2008.90
\n🔍 SAMPLE PREDICTIONS
      id        price
0  20000  1591.245606
1  20001  2841.417280
2  20002   989.742944
3  20003   923.882815
4  20004  1274.751776
5  20005  1130.139266
6  20006  2337.987224
7  20007  4544.325464
8  20008  4571.125922
9  20009  4488.462465
\n💾 SAVING MODELS
  Saved: ultra-lightgbm-r2-0.6780.joblib
\n🏆 OPTIMIZATION COMPLETE!
Achieved R² = 0.6780
Target was R² ≥ 0.68
🎯 Very close! Only 0.0020 away from target.


In [19]:
# COMPLETE OPTIMIZED DIAMOND PRICE PREDICTION MODEL - R² = 0.6780
# This cell contains all the code that achieved the 0.6780 R² score in one complete block

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')

print("🚀 COMPLETE OPTIMIZED DIAMOND PRICE PREDICTION MODEL")
print("="*60)


print("📊 Loading and preprocessing data...")

# Load fresh data
train_data = pd.read_csv('/home/ayushz/Kaggle/Datasets/train.csv')
test_data = pd.read_csv('/home/ayushz/Kaggle/Datasets/test.csv')

print(f"Original train shape: {train_data.shape}, test shape: {test_data.shape}")

# Advanced outlier removal using IQR method
def remove_outliers_iqr(df, column, factor=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove price outliers
original_len = len(train_data)
train_data = remove_outliers_iqr(train_data, 'price', factor=1.2)
print(f"Removed {original_len - len(train_data)} price outliers")

# Remove dimension outliers
for dim in ['x', 'y', 'z']:
    train_data = remove_outliers_iqr(train_data, dim, factor=2.0)

print(f"Final training data shape: {train_data.shape}")

# Advanced missing value handling
for col in ['x','y','z']:
    # Replace very small and zero values with NaN
    train_data.loc[train_data[col] <= 0.01, col] = np.nan
    test_data.loc[test_data[col] <= 0.01, col] = np.nan
    
    # Group-based imputation by cut and clarity
    for cut_val in train_data['cut'].unique():
        for clarity_val in train_data['clarity'].unique():
            mask_train = (train_data['cut'] == cut_val) & (train_data['clarity'] == clarity_val)
            mask_test = (test_data['cut'] == cut_val) & (test_data['clarity'] == clarity_val)
            
            if mask_train.sum() > 0 and train_data.loc[mask_train, col].notna().sum() > 0:
                median_val = train_data.loc[mask_train, col].median()
                train_data.loc[mask_train & train_data[col].isna(), col] = median_val
                test_data.loc[mask_test & test_data[col].isna(), col] = median_val
    
    # Fill remaining with overall median
    overall_median = train_data[col].median()
    train_data[col].fillna(overall_median, inplace=True)
    test_data[col].fillna(overall_median, inplace=True)

# Clean categorical variables
train_data['cut'] = train_data['cut'].str.strip()
test_data['cut'] = test_data['cut'].str.strip()

print("✅ Data preprocessing completed!")


print("🔧 Creating advanced features...")

# Consistent label encoding
le_cut = LabelEncoder()
le_color = LabelEncoder()
le_clarity = LabelEncoder()

# Fit on combined data for consistency
all_data = pd.concat([
    train_data[['cut', 'color', 'clarity']], 
    test_data[['cut', 'color', 'clarity']]
])

le_cut.fit(all_data['cut'])
le_color.fit(all_data['color'])
le_clarity.fit(all_data['clarity'])

train_data['cut_encoded'] = le_cut.transform(train_data['cut'])
train_data['color_encoded'] = le_color.transform(train_data['color'])
train_data['clarity_encoded'] = le_clarity.transform(train_data['clarity'])

test_data['cut_encoded'] = le_cut.transform(test_data['cut'])
test_data['color_encoded'] = le_color.transform(test_data['color'])
test_data['clarity_encoded'] = le_clarity.transform(test_data['clarity'])

# Basic geometric features
train_data['volume'] = train_data['x'] * train_data['y'] * train_data['z']
test_data['volume'] = test_data['x'] * test_data['y'] * test_data['z']

train_data['surface_area'] = 2 * (train_data['x']*train_data['y'] + 
                                 train_data['x']*train_data['z'] + 
                                 train_data['y']*train_data['z'])
test_data['surface_area'] = 2 * (test_data['x']*test_data['y'] + 
                                test_data['x']*test_data['z'] + 
                                test_data['y']*test_data['z'])

# Advanced ratios and transformations
train_data['carat_per_volume'] = train_data['carat'] / (train_data['volume'] + 1e-8)
test_data['carat_per_volume'] = test_data['carat'] / (test_data['volume'] + 1e-8)

train_data['density'] = train_data['carat'] / (train_data['volume'] + 1e-8)
test_data['density'] = test_data['carat'] / (test_data['volume'] + 1e-8)

train_data['aspect_xy'] = train_data['x'] / (train_data['y'] + 1e-8)
test_data['aspect_xy'] = test_data['x'] / (test_data['y'] + 1e-8)

train_data['aspect_xz'] = train_data['x'] / (train_data['z'] + 1e-8)
test_data['aspect_xz'] = test_data['x'] / (test_data['z'] + 1e-8)

# Domain knowledge quality scores
cut_quality = {'Fair': 1, 'Good': 2, 'Very Good': 3, 'Premium': 4, 'Ideal': 5}
color_quality = {'J': 1, 'I': 2, 'H': 3, 'G': 4, 'F': 5, 'E': 6, 'D': 7}
clarity_quality = {'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2': 4, 'VS1': 5, 'VVS2': 6, 'VVS1': 7, 'IF': 8}

train_data['cut_quality'] = train_data['cut'].map(cut_quality).fillna(3)
test_data['cut_quality'] = test_data['cut'].map(cut_quality).fillna(3)

train_data['color_quality'] = train_data['color'].map(color_quality).fillna(4)
test_data['color_quality'] = test_data['color'].map(color_quality).fillna(4)

train_data['clarity_quality'] = train_data['clarity'].map(clarity_quality).fillna(4)
test_data['clarity_quality'] = test_data['clarity'].map(clarity_quality).fillna(4)

train_data['overall_quality'] = (train_data['cut_quality'] + 
                                train_data['color_quality'] + 
                                train_data['clarity_quality'])
test_data['overall_quality'] = (test_data['cut_quality'] + 
                               test_data['color_quality'] + 
                               test_data['clarity_quality'])

# Log transformations
train_data['log_carat'] = np.log1p(train_data['carat'])
test_data['log_carat'] = np.log1p(test_data['carat'])

train_data['log_volume'] = np.log1p(train_data['volume'])
test_data['log_volume'] = np.log1p(test_data['volume'])

# Polynomial features
for col in ['carat', 'volume', 'overall_quality']:
    train_data[f'{col}_squared'] = train_data[col] ** 2
    test_data[f'{col}_squared'] = test_data[col] ** 2
    
    train_data[f'{col}_cubed'] = train_data[col] ** 3
    test_data[f'{col}_cubed'] = test_data[col] ** 3

# Interaction features
train_data['carat_quality'] = train_data['carat'] * train_data['overall_quality']
test_data['carat_quality'] = test_data['carat'] * test_data['overall_quality']

train_data['volume_quality'] = train_data['volume'] * train_data['overall_quality']
test_data['volume_quality'] = test_data['volume'] * test_data['overall_quality']

# Binning features
train_data['carat_bin'] = pd.cut(train_data['carat'], bins=10, labels=False)
test_data['carat_bin'] = pd.cut(test_data['carat'], bins=10, labels=False)

print("✅ Advanced feature engineering completed!")


print("🎯 Training ultra-optimized LightGBM model...")

# Feature selection
feature_cols = [
    'carat', 'cut_encoded', 'color_encoded', 'clarity_encoded', 'depth', 'table',
    'x', 'y', 'z', 'volume', 'surface_area', 'carat_per_volume', 'density',
    'aspect_xy', 'aspect_xz', 'cut_quality', 'color_quality', 'clarity_quality',
    'overall_quality', 'log_carat', 'log_volume', 'carat_squared', 'volume_squared',
    'overall_quality_squared', 'carat_cubed', 'volume_cubed', 'overall_quality_cubed',
    'carat_quality', 'volume_quality', 'carat_bin'
]

X = train_data[feature_cols]
y = train_data['price']

print(f"Feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, 
    stratify=pd.cut(y, bins=10, labels=False)
)

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")

# Ultra-optimized LightGBM parameters (achieved R² = 0.6780)
params_ultra_lgb = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 300,          # High complexity
    'max_depth': 15,            # Deep trees
    'learning_rate': 0.01,      # Low learning rate for stability
    'feature_fraction': 0.95,   # Use most features
    'bagging_fraction': 0.95,   # Use most data
    'bagging_freq': 5,
    'min_child_samples': 5,     # Allow small leaves
    'min_split_gain': 0.01,     # Easy splitting
    'reg_alpha': 0.01,          # Minimal regularization
    'reg_lambda': 0.01,
    'subsample': 0.95,
    'colsample_bytree': 0.95,
    'max_bin': 500,             # High granularity
    'verbose': -1,
    'random_state': 42,
    'extra_trees': True,        # Extra randomization
    'force_row_wise': True,
    'device': 'cpu',
    'num_threads': -1
}

# Create datasets
train_lgb = lgb.Dataset(X_train, label=y_train)
valid_lgb = lgb.Dataset(X_test, label=y_test, reference=train_lgb)

# Train the model
model_final = lgb.train(
    params_ultra_lgb,
    train_lgb,
    valid_sets=[valid_lgb],
    num_boost_round=8000,       # High iterations
    callbacks=[
        lgb.early_stopping(300), # High patience
        lgb.log_evaluation(100)   # Show progress
    ]
)

# Evaluate the model
y_pred = model_final.predict(X_test, num_iteration=model_final.best_iteration)
r2_final = r2_score(y_test, y_pred)
mae_final = mean_absolute_error(y_test, y_pred)

print(f"🎉 FINAL MODEL PERFORMANCE:")
print(f"   R² Score: {r2_final:.4f}")
print(f"   MAE: {mae_final:.2f}")
print(f"   Best iteration: {model_final.best_iteration}")


print("📝 Generating final predictions...")

# Prepare test data
X_test_final = test_data[feature_cols]

# Generate predictions
final_predictions = model_final.predict(X_test_final, num_iteration=model_final.best_iteration)

# Create submission
submission_final = pd.DataFrame({
    'id': test_data['id'],
    'price': final_predictions
})

# Ensure positive prices
submission_final['price'] = submission_final['price'].clip(lower=0)

# Save files
submission_file = 'diamond_prediction_r2_0678.csv'
model_file = 'ultra_lightgbm_r2_0678.joblib'

submission_final.to_csv(submission_file, index=False)
joblib.dump(model_final, model_file)

print(f"💾 FILES SAVED:")
print(f"   Submission: {submission_file}")
print(f"   Model: {model_file}")

print(f"\\n📊 PREDICTION STATISTICS:")
print(f"   Min price: ${submission_final['price'].min():.2f}")
print(f"   Max price: ${submission_final['price'].max():.2f}")
print(f"   Mean price: ${submission_final['price'].mean():.2f}")
print(f"   Median price: ${submission_final['price'].median():.2f}")

print("\\n" + "="*60)
print("🏆 COMPLETE MODEL EXECUTION FINISHED!")
print(f"🎯 Achieved R² Score: {r2_final:.4f}")
print(f"📁 Output: {submission_file}")
print("="*60)

🚀 COMPLETE OPTIMIZED DIAMOND PRICE PREDICTION MODEL
📊 Loading and preprocessing data...
Original train shape: (20000, 11), test shape: (30000, 10)
Removed 1758 price outliers
Final training data shape: (18239, 11)
✅ Data preprocessing completed!
🔧 Creating advanced features...
✅ Advanced feature engineering completed!
🎯 Training ultra-optimized LightGBM model...
Feature matrix shape: (18239, 30)
Target shape: (18239,)
Train set: (14591, 30), Test set: (3648, 30)
Training until validation scores don't improve for 300 rounds
✅ Data preprocessing completed!
🔧 Creating advanced features...
✅ Advanced feature engineering completed!
🎯 Training ultra-optimized LightGBM model...
Feature matrix shape: (18239, 30)
Target shape: (18239,)
Train set: (14591, 30), Test set: (3648, 30)
Training until validation scores don't improve for 300 rounds
[100]	valid_0's rmse: 1609.04
[100]	valid_0's rmse: 1609.04
[200]	valid_0's rmse: 1443.63
[200]	valid_0's rmse: 1443.63
[300]	valid_0's rmse: 1420.21
[300]	

In [3]:
# Improved Diamond Price Prediction Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/ayushz/Kaggle/Datasets/train.csv')
test = pd.read_csv('/home/ayushz/Kaggle/Datasets/test.csv')

print("Original train shape:", train.shape)
print("Original test shape:", test.shape)

# Better outlier detection and handling
def remove_outliers(df, columns, z_threshold=3):
    """Remove outliers using Z-score method"""
    for col in columns:
        if col in df.columns:
            z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
            df = df[z_scores < z_threshold]
    return df

# Remove price outliers from training data
train_clean = remove_outliers(train, ['price'], z_threshold=3)
print(f"Removed {len(train) - len(train_clean)} outliers from training data")
train = train_clean.copy()

# Improved missing value handling for dimensions
for col in ['x','y','z']:
    # Replace 0s and very small values with NaN
    train.loc[train[col] <= 0.01, col] = np.nan
    test.loc[test[col] <= 0.01, col] = np.nan
    
    # Fill with median grouped by cut and carat quartiles
    train['carat_quartile'] = pd.qcut(train['carat'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    test['carat_quartile'] = pd.qcut(test['carat'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    
    for cut in train['cut'].unique():
        for quartile in train['carat_quartile'].unique():
            mask_train = (train['cut'] == cut) & (train['carat_quartile'] == quartile)
            mask_test = (test['cut'] == cut) & (test['carat_quartile'] == quartile)
            
            if mask_train.sum() > 0:
                median_val = train.loc[mask_train, col].median()
                if pd.isna(median_val):
                    median_val = train[col].median()
                train.loc[mask_train & train[col].isna(), col] = median_val
                test.loc[mask_test & test[col].isna(), col] = median_val

# Fill remaining NaN values with overall median
for col in ['x','y','z']:
    train[col].fillna(train[col].median(), inplace=True)
    test[col].fillna(train[col].median(), inplace=True)

# Clean categorical variables
train['cut'] = train['cut'].str.strip()
test['cut'] = test['cut'].str.strip()

# Consistent label encoding
le_cut = LabelEncoder()
le_color = LabelEncoder()
le_clarity = LabelEncoder()

# Fit on combined data to ensure consistency
all_cuts = list(train['cut'].unique()) + list(test['cut'].unique())
all_colors = list(train['color'].unique()) + list(test['color'].unique())
all_clarity = list(train['clarity'].unique()) + list(test['clarity'].unique())

le_cut.fit(list(set(all_cuts)))
le_color.fit(list(set(all_colors)))
le_clarity.fit(list(set(all_clarity)))

train['cut_encoded'] = le_cut.transform(train['cut'])
train['color_encoded'] = le_color.transform(train['color'])
train['clarity_encoded'] = le_clarity.transform(train['clarity'])

test['cut_encoded'] = le_cut.transform(test['cut'])
test['color_encoded'] = le_color.transform(test['color'])
test['clarity_encoded'] = le_clarity.transform(test['clarity'])

print("Data preprocessing completed successfully!")

Original train shape: (20000, 11)
Original test shape: (30000, 10)
Removed 450 outliers from training data
Data preprocessing completed successfully!


In [4]:
# Enhanced Feature Engineering
# Basic volume and ratios
train['volume'] = train['x'] * train['y'] * train['z']
test['volume'] = test['x'] * test['y'] * test['z']

# Carat per volume ratio
train['carat_per_volume'] = train['carat'] / (train['volume'] + 1e-9)
test['carat_per_volume'] = test['carat'] / (test['volume'] + 1e-9)

# Additional geometric features
train['surface_area'] = 2 * (train['x']*train['y'] + train['x']*train['z'] + train['y']*train['z'])
test['surface_area'] = 2 * (test['x']*test['y'] + test['x']*test['z'] + test['y']*test['z'])

train['aspect_ratio_xy'] = train['x'] / (train['y'] + 1e-9)
test['aspect_ratio_xy'] = test['x'] / (test['y'] + 1e-9)

train['aspect_ratio_xz'] = train['x'] / (train['z'] + 1e-9)
test['aspect_ratio_xz'] = test['x'] / (test['z'] + 1e-9)

# Depth and table ratios (important for diamond quality)
train['depth_ratio'] = train['depth'] / 100.0
test['depth_ratio'] = test['depth'] / 100.0

train['table_ratio'] = train['table'] / 100.0
test['table_ratio'] = test['table'] / 100.0

# Carat interactions with categorical features
train['carat_cut_interaction'] = train['carat'] * train['cut_encoded']
test['carat_cut_interaction'] = test['carat'] * test['cut_encoded']

train['carat_color_interaction'] = train['carat'] * train['color_encoded']
test['carat_color_interaction'] = test['carat'] * test['color_encoded']

train['carat_clarity_interaction'] = train['carat'] * train['clarity_encoded']
test['carat_clarity_interaction'] = test['carat'] * test['clarity_encoded']

# Quality score based on cut, color, clarity
train['quality_score'] = train['cut_encoded'] + train['color_encoded'] + train['clarity_encoded']
test['quality_score'] = test['cut_encoded'] + test['color_encoded'] + test['clarity_encoded']

# Volume to carat ratio (density indicator)
train['density'] = train['carat'] / (train['volume'] + 1e-9)
test['density'] = test['carat'] / (test['volume'] + 1e-9)

# Price per carat for training (useful for understanding patterns)
train['price_per_carat'] = train['price'] / train['carat']

print("Enhanced feature engineering completed!")
print(f"Training features shape: {train.shape}")
print(f"Test features shape: {test.shape}")

Enhanced feature engineering completed!
Training features shape: (19550, 28)
Test features shape: (30000, 26)


In [5]:
# Prepare features for modeling (excluding unnecessary columns)
feature_columns = [
    'carat', 'cut_encoded', 'color_encoded', 'clarity_encoded', 'depth', 'table',
    'x', 'y', 'z', 'volume', 'carat_per_volume', 'surface_area', 
    'aspect_ratio_xy', 'aspect_ratio_xz', 'depth_ratio', 'table_ratio',
    'carat_cut_interaction', 'carat_color_interaction', 'carat_clarity_interaction',
    'quality_score', 'density'
]

X = train[feature_columns]
y = train['price']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

# Create LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Optimized hyperparameters for better performance
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 100,           # Increased from 65
    'max_depth': 7,              # Increased from 3
    'learning_rate': 0.05,       # Decreased for better generalization
    'subsample': 0.8,            # Slightly decreased
    'colsample_bytree': 0.8,     # Increased from 0.7
    'min_child_samples': 20,     # Decreased from 55
    'min_split_gain': 0.1,       # Decreased from 0.2222
    'reg_alpha': 0.1,            # Decreased from 0.2222
    'reg_lambda': 0.1,           # Increased from 0.0
    'max_bin': 255,              # Increased from 250
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'force_row_wise': True,
    'random_state': 42
}

print("Training optimized LightGBM model...")

# Train the model with early stopping
model_lgb_optimized = lgb.train(
    params,
    train_data,
    valid_sets=[test_data],
    num_boost_round=2000,        # Increased from 1000
    callbacks=[
        lgb.early_stopping(100),  # Increased patience
        lgb.log_evaluation(200)
    ]
)

# Predict and evaluate
y_pred_optimized = model_lgb_optimized.predict(X_test, num_iteration=model_lgb_optimized.best_iteration)
mae_optimized = mean_absolute_error(y_test, y_pred_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

print(f"Optimized LightGBM MAE: {mae_optimized:.2f}")
print(f"Optimized LightGBM R² Score: {r2_optimized:.4f}")

# Feature importance
feature_importance = model_lgb_optimized.feature_importance(importance_type='gain')
feature_names = X_train.columns
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(importance_df.head(10))

Training set shape: (15640, 21)
Test set shape: (3910, 21)
Training optimized LightGBM model...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[63]	valid_0's rmse: 2014.04
Optimized LightGBM MAE: 1231.25
Optimized LightGBM R² Score: 0.6505

Top 10 Most Important Features:
                      feature    importance
0                       carat  6.509195e+11
9                      volume  1.745835e+11
6                           x  1.465852e+11
11               surface_area  3.833265e+10
8                           z  3.600991e+10
7                           y  3.493062e+10
13            aspect_ratio_xz  1.204206e+10
10           carat_per_volume  1.177935e+10
18  carat_clarity_interaction  1.168009e+10
4                       depth  1.105688e+10


In [ ]:
# Generate improved submission
print("Generating improved submission...")

# Prepare test data for submission
X_submission = test[feature_columns]

# Predict on the competition test set
competition_predictions_optimized = model_lgb_optimized.predict(
    X_submission, 
    num_iteration=model_lgb_optimized.best_iteration
)

# Create submission DataFrame
submission_df_optimized = pd.DataFrame({
    'id': test['id'],
    'price': competition_predictions_optimized
})

# Ensure prices are not negative
submission_df_optimized['price'] = submission_df_optimized['price'].clip(0)

# Save the improved submission file
submission_df_optimized.to_csv('improved_submission.csv', index=False)

print("Improved submission file 'improved_submission.csv' created successfully!")
print(f"R² Score achieved: {r2_optimized:.4f}")
print(f"Target R² Score: 0.70")
print(f"Goal achieved: {'YES' if r2_optimized >= 0.70 else 'NO'}")

print("\nSubmission file preview:")
print(submission_df_optimized.head(10))

print(f"\nSubmission statistics:")
print(f"Min price: ${submission_df_optimized['price'].min():.2f}")
print(f"Max price: ${submission_df_optimized['price'].max():.2f}")
print(f"Mean price: ${submission_df_optimized['price'].mean():.2f}")
print(f"Median price: ${submission_df_optimized['price'].median():.2f}")

# Save the model
import joblib
joblib.dump(model_lgb_optimized, 'improved_lgbm_model.joblib')
print("Improved model saved as 'improved_lgbm_model.joblib'!")

In [7]:
# Let's try XGBoost and Random Forest ensemble for better performance
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

print("Training ensemble models for improved performance...")

# XGBoost model
xgb_params = {
    'max_depth': 8,
    'learning_rate': 0.05,
    'n_estimators': 500,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,
    'n_jobs': -1
}

model_xgb = xgb.XGBRegressor(**xgb_params)
model_xgb.fit(X_train, y_train)

y_pred_xgb = model_xgb.predict(X_test)
r2_xgb = r2_score(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)

print(f"XGBoost R² Score: {r2_xgb:.4f}")
print(f"XGBoost MAE: {mae_xgb:.2f}")

# Random Forest model
rf_params = {
    'n_estimators': 200,
    'max_depth': 15,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'random_state': 42,
    'n_jobs': -1
}

model_rf = RandomForestRegressor(**rf_params)
model_rf.fit(X_train, y_train)

y_pred_rf = model_rf.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print(f"Random Forest R² Score: {r2_rf:.4f}")
print(f"Random Forest MAE: {mae_rf:.2f}")

# Ensemble prediction (weighted average)
# Weight models based on their R² performance
weights = [r2_optimized, r2_xgb, r2_rf]
weights = np.array(weights) / np.sum(weights)

print(f"Model weights: LGB={weights[0]:.3f}, XGB={weights[1]:.3f}, RF={weights[2]:.3f}")

y_pred_ensemble = (weights[0] * y_pred_optimized + 
                  weights[1] * y_pred_xgb + 
                  weights[2] * y_pred_rf)

r2_ensemble = r2_score(y_test, y_pred_ensemble)
mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)

print(f"Ensemble R² Score: {r2_ensemble:.4f}")
print(f"Ensemble MAE: {mae_ensemble:.2f}")

# Choose the best model
models = {
    'LightGBM': (model_lgb_optimized, r2_optimized, y_pred_optimized),
    'XGBoost': (model_xgb, r2_xgb, y_pred_xgb),
    'Random Forest': (model_rf, r2_rf, y_pred_rf),
    'Ensemble': (None, r2_ensemble, y_pred_ensemble)
}

best_model_name = max(models.keys(), key=lambda x: models[x][1])
best_r2 = models[best_model_name][1]

print(f"\nBest model: {best_model_name} with R² = {best_r2:.4f}")
print(f"Target achieved (R² ≥ 0.70): {'YES' if best_r2 >= 0.70 else 'NO'}")

Training ensemble models for improved performance...
XGBoost R² Score: 0.6163
XGBoost MAE: 1300.52
Random Forest R² Score: 0.6424
Random Forest MAE: 1243.63
Model weights: LGB=0.341, XGB=0.323, RF=0.336
Ensemble R² Score: 0.6440
Ensemble MAE: 1242.68

Best model: LightGBM with R² = 0.6505
Target achieved (R² ≥ 0.70): NO


In [ ]:
# Generate final submission with best performing model
print("Generating final optimized submission...")

# Get predictions from all models on test set
X_submission = test[feature_columns]

lgb_submission_pred = model_lgb_optimized.predict(X_submission, num_iteration=model_lgb_optimized.best_iteration)
xgb_submission_pred = model_xgb.predict(X_submission)
rf_submission_pred = model_rf.predict(X_submission)

# Ensemble prediction for submission
final_predictions = (weights[0] * lgb_submission_pred + 
                    weights[1] * xgb_submission_pred + 
                    weights[2] * rf_submission_pred)

# Create final submission
final_submission_df = pd.DataFrame({
    'id': test['id'],
    'price': final_predictions
})

# Ensure prices are not negative
final_submission_df['price'] = final_submission_df['price'].clip(0)

# Save the final submission
final_submission_df.to_csv('final_optimized_submission.csv', index=False)

print("Final optimized submission saved as 'final_optimized_submission.csv'")
print(f"Best R² Score achieved: {best_r2:.4f}")
print(f"Target R² (≥ 0.70): {'✓ ACHIEVED' if best_r2 >= 0.70 else '✗ NOT ACHIEVED'}")

print("\nFinal submission preview:")
print(final_submission_df.head())

print(f"\nFinal submission statistics:")
print(f"Min price: ${final_submission_df['price'].min():.2f}")
print(f"Max price: ${final_submission_df['price'].max():.2f}")
print(f"Mean price: ${final_submission_df['price'].mean():.2f}")
print(f"Median price: ${final_submission_df['price'].median():.2f}")

# Save all models
import joblib
joblib.dump(model_xgb, 'xgb_model_optimized.joblib')
joblib.dump(model_rf, 'rf_model_optimized.joblib')

print("\nAll models saved successfully!")
print("- improved_lgbm_model.joblib")
print("- xgb_model_optimized.joblib") 
print("- rf_model_optimized.joblib")

In [8]:
# Final aggressive optimization attempt
print("Attempting final aggressive optimization...")

# Try polynomial features for key variables
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features for the most important features
poly_features = ['carat', 'x', 'y', 'z', 'volume']
X_train_poly = X_train.copy()
X_test_poly = X_test.copy()

for feature in poly_features:
    X_train_poly[f'{feature}_squared'] = X_train[feature] ** 2
    X_test_poly[f'{feature}_squared'] = X_test[feature] ** 2
    X_train_poly[f'{feature}_cubed'] = X_train[feature] ** 3
    X_test_poly[f'{feature}_cubed'] = X_test[feature] ** 3

# Add more interaction features
X_train_poly['carat_volume_interaction'] = X_train['carat'] * X_train['volume']
X_test_poly['carat_volume_interaction'] = X_test['carat'] * X_test['volume']

X_train_poly['xyz_product'] = X_train['x'] * X_train['y'] * X_train['z']
X_test_poly['xyz_product'] = X_test['x'] * X_test['y'] * X_test['z']

print(f"Enhanced feature set size: {X_train_poly.shape[1]} features")

# Create new LightGBM datasets with polynomial features
train_data_poly = lgb.Dataset(X_train_poly, label=y_train)
test_data_poly = lgb.Dataset(X_test_poly, label=y_test, reference=train_data_poly)

# Ultra-optimized parameters
params_ultra = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 150,          # Increased further
    'max_depth': 10,            # Increased
    'learning_rate': 0.03,      # Decreased for more iterations
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'min_child_samples': 10,    # Decreased
    'min_split_gain': 0.05,     # Decreased
    'reg_alpha': 0.05,          # Decreased
    'reg_lambda': 0.05,         # Decreased
    'max_bin': 300,             # Increased
    'feature_fraction': 0.85,
    'bagging_fraction': 0.85,
    'bagging_freq': 7,
    'verbose': -1,
    'force_row_wise': True,
    'random_state': 42,
    'extra_trees': True         # Enable extra randomization
}

print("Training ultra-optimized model...")
model_ultra = lgb.train(
    params_ultra,
    train_data_poly,
    valid_sets=[test_data_poly],
    num_boost_round=3000,       # Much higher
    callbacks=[
        lgb.early_stopping(150), # Increased patience
        lgb.log_evaluation(300)
    ]
)

y_pred_ultra = model_ultra.predict(X_test_poly, num_iteration=model_ultra.best_iteration)
r2_ultra = r2_score(y_test, y_pred_ultra)
mae_ultra = mean_absolute_error(y_test, y_pred_ultra)

print(f"Ultra-optimized R² Score: {r2_ultra:.4f}")
print(f"Ultra-optimized MAE: {mae_ultra:.2f}")
print(f"Target achieved (R² ≥ 0.70): {'YES ✓' if r2_ultra >= 0.70 else 'NO ✗'}")

if r2_ultra > best_r2:
    best_r2 = r2_ultra
    best_model_name = "Ultra-optimized LightGBM"
    print(f"New best model: {best_model_name} with R² = {best_r2:.4f}")

Attempting final aggressive optimization...
Enhanced feature set size: 33 features
Training ultra-optimized model...
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[122]	valid_0's rmse: 2006.9
Ultra-optimized R² Score: 0.6529
Ultra-optimized MAE: 1222.96
Target achieved (R² ≥ 0.70): NO ✗
New best model: Ultra-optimized LightGBM with R² = 0.6529


In [9]:
# Generate final submission with the best performing model
print("Generating final optimized submission...")

# Prepare enhanced test data
X_submission_enhanced = test[feature_columns].copy()

# Add polynomial features to submission data
for feature in poly_features:
    if feature in X_submission_enhanced.columns:
        X_submission_enhanced[f'{feature}_squared'] = X_submission_enhanced[feature] ** 2
        X_submission_enhanced[f'{feature}_cubed'] = X_submission_enhanced[feature] ** 3

# Add interaction features to submission data
X_submission_enhanced['carat_volume_interaction'] = X_submission_enhanced['carat'] * X_submission_enhanced['volume']
X_submission_enhanced['xyz_product'] = X_submission_enhanced['x'] * X_submission_enhanced['y'] * X_submission_enhanced['z']

# Use the best performing model for final predictions
if r2_ultra >= 0.70:
    final_predictions = model_ultra.predict(X_submission_enhanced, num_iteration=model_ultra.best_iteration)
    model_used = "Ultra-optimized LightGBM"
    final_r2 = r2_ultra
else:
    # If ultra model doesn't achieve 0.70, use the best available model
    final_predictions = model_lgb_optimized.predict(test[feature_columns], num_iteration=model_lgb_optimized.best_iteration)
    model_used = "Optimized LightGBM"
    final_r2 = r2_optimized

# Create final submission
final_submission = pd.DataFrame({
    'id': test['id'],
    'price': final_predictions
})

# Ensure prices are positive
final_submission['price'] = final_submission['price'].clip(0)

# Save the final submission
final_submission.to_csv('final_diamond_predictions.csv', index=False)

print("="*60)
print("FINAL RESULTS")
print("="*60)
print(f"Model used: {model_used}")
print(f"Final R² Score: {final_r2:.4f}")
print(f"Target R² (≥ 0.70): {'✓ ACHIEVED' if final_r2 >= 0.70 else '✗ NOT ACHIEVED'}")
print(f"CSV saved as: final_diamond_predictions.csv")
print("="*60)

print("\nSubmission preview:")
print(final_submission.head())

print(f"\nPrediction statistics:")
print(f"Min price: ${final_submission['price'].min():.2f}")
print(f"Max price: ${final_submission['price'].max():.2f}")
print(f"Mean price: ${final_submission['price'].mean():.2f}")
print(f"Median price: ${final_submission['price'].median():.2f}")
print(f"Total predictions: {len(final_submission)}")

# Save the best model
import joblib
if r2_ultra >= 0.70:
    joblib.dump(model_ultra, 'ultra_optimized_model.joblib')
    print("\nUltra-optimized model saved as 'ultra_optimized_model.joblib'")
else:
    print("\nUsing previously saved 'improved_lgbm_model.joblib'")

Generating final optimized submission...
FINAL RESULTS
Model used: Optimized LightGBM
Final R² Score: 0.6505
Target R² (≥ 0.70): ✗ NOT ACHIEVED
CSV saved as: final_diamond_predictions.csv

Submission preview:
      id        price
0  20000  1735.985183
1  20001  3188.714816
2  20002  1097.940952
3  20003  1125.660944
4  20004  1238.661036

Prediction statistics:
Min price: $828.39
Max price: $14312.27
Mean price: $3772.14
Median price: $2670.91
Total predictions: 30000

Using previously saved 'improved_lgbm_model.joblib'


In [20]:
# === Anti-Overfitting Model for Better Kaggle Performance ===
# Goal: Create a model that generalizes better (less gap between validation and Kaggle)

import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error
import lightgbm as lgb

print("🎯 Creating Anti-Overfitting LightGBM Model")
print("=" * 60)

# Conservative parameters to prevent overfitting
params_conservative = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,  # Much smaller (was 300)
    'max_depth': 6,    # Smaller depth (was 15)  
    'learning_rate': 0.05,  # Higher learning rate (was 0.01)
    'feature_fraction': 0.8,  # Add feature sampling
    'bagging_fraction': 0.8,  # Add row sampling
    'bagging_freq': 5,
    'min_child_samples': 20,  # Increase minimum samples
    'min_child_weight': 0.001,
    'reg_alpha': 1.0,  # L1 regularization
    'reg_lambda': 1.0, # L2 regularization
    'random_state': 42,
    'verbosity': -1
}

# Use cross-validation for more robust evaluation
print("📊 Performing 5-fold Cross-Validation...")
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_r2_scores = []
cv_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_advanced)):
    print(f"\nFold {fold + 1}/5:")
    
    # Split data
    X_fold_train, X_fold_val = X_advanced.iloc[train_idx], X_advanced.iloc[val_idx]
    y_fold_train, y_fold_val = y_advanced.iloc[train_idx], y_advanced.iloc[val_idx]
    
    # Create datasets
    train_data_fold = lgb.Dataset(X_fold_train, label=y_fold_train)
    val_data_fold = lgb.Dataset(X_fold_val, label=y_fold_val, reference=train_data_fold)
    
    # Train with early stopping
    model_fold = lgb.train(
        params_conservative,
        train_data_fold,
        valid_sets=[val_data_fold],
        num_boost_round=1000,  # Much fewer rounds (was 8000)
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(0)
        ]
    )
    
    # Validate
    y_pred_fold = model_fold.predict(X_fold_val, num_iteration=model_fold.best_iteration)
    r2_fold = r2_score(y_fold_val, y_pred_fold)
    
    cv_r2_scores.append(r2_fold)
    cv_models.append(model_fold)
    
    print(f"  R² Score: {r2_fold:.4f}")

print(f"\n📈 Cross-Validation Results:")
print(f"  Mean R²: {np.mean(cv_r2_scores):.4f} ± {np.std(cv_r2_scores):.4f}")
print(f"  Individual folds: {[f'{score:.4f}' for score in cv_r2_scores]}")

# Train final model on all data with conservative parameters
print(f"\n🔧 Training Final Conservative Model...")
train_data_final = lgb.Dataset(X_advanced, label=y_advanced)

model_conservative = lgb.train(
    params_conservative,
    train_data_final,
    num_boost_round=int(np.mean([model.best_iteration for model in cv_models])),
    callbacks=[lgb.log_evaluation(0)]
)

print(f"  Final model trained with {model_conservative.num_trees} trees")

# Generate predictions for submission
print(f"\n🎲 Generating Conservative Predictions...")
conservative_predictions = model_conservative.predict(X_submission_enhanced, num_iteration=model_conservative.best_iteration)

# Create submission file
conservative_submission = pd.DataFrame({
    'id': range(len(conservative_predictions)),
    'price': conservative_predictions
})

# Save conservative model and predictions
conservative_filename = f"diamond_conservative_cv_{np.mean(cv_r2_scores):.4f}.csv"
conservative_model_file = f"conservative_lgb_cv_{np.mean(cv_r2_scores):.4f}.joblib"

conservative_submission.to_csv(conservative_filename, index=False)
joblib.dump(model_conservative, conservative_model_file)

print(f"✅ Conservative Model Results:")
print(f"  Cross-validation R²: {np.mean(cv_r2_scores):.4f} ± {np.std(cv_r2_scores):.4f}")
print(f"  Submission saved: {conservative_filename}")
print(f"  Model saved: {conservative_model_file}")
print(f"  This model should generalize better to Kaggle!")

# Compare with previous ultra model
print(f"\n⚖️ Model Comparison:")
print(f"  Ultra Model (validation): {r2_ultra_lgb:.4f} → Kaggle: 0.6188 (Gap: {r2_ultra_lgb - 0.6188:.4f})")
print(f"  Conservative Model (CV): {np.mean(cv_r2_scores):.4f} → Expected Kaggle: ~{np.mean(cv_r2_scores):.4f}")
print(f"  Expected improvement in Kaggle score: {np.mean(cv_r2_scores) - 0.6188:.4f}")

🎯 Creating Anti-Overfitting LightGBM Model
📊 Performing 5-fold Cross-Validation...

Fold 1/5:
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[62]	valid_0's rmse: 1389.77
  R² Score: 0.6819

Fold 2/5:
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[64]	valid_0's rmse: 1444.87
  R² Score: 0.6549

Fold 3/5:
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[65]	valid_0's rmse: 1461.3
  R² Score: 0.6613

Fold 4/5:
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[68]	valid_0's rmse: 1445.66
  R² Score: 0.6645

Fold 5/5:
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[74]	valid_0's rmse: 1441.21
  R² Score: 0.6813

📈 Cross-Validation Results:
  Mean R²: 0.6688 ± 0.0109
  Individual folds: ['0.6819', '0.6549', '0.6613', '0.6645', '0.6813']

🔧 Training Fina

[LightGBM] [Fatal] The number of features in data (33) is not the same as it was in training data (30).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.


LightGBMError: The number of features in data (33) is not the same as it was in training data (30).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [21]:
# Debug feature mismatch
print("🔍 Feature Mismatch Debug:")
print(f"X_advanced shape: {X_advanced.shape}")
print(f"X_submission_enhanced shape: {X_submission_enhanced.shape}")
print(f"X_advanced columns: {len(X_advanced.columns)}")
print(f"X_submission_enhanced columns: {len(X_submission_enhanced.columns)}")

print(f"\nX_advanced columns:")
for i, col in enumerate(X_advanced.columns):
    print(f"  {i+1}. {col}")

print(f"\nX_submission_enhanced columns:")
for i, col in enumerate(X_submission_enhanced.columns):
    print(f"  {i+1}. {col}")

# Check which columns are missing
missing_in_submission = set(X_advanced.columns) - set(X_submission_enhanced.columns)
extra_in_submission = set(X_submission_enhanced.columns) - set(X_advanced.columns)

print(f"\nMissing in submission: {missing_in_submission}")
print(f"Extra in submission: {extra_in_submission}")

# Use the same features for both
common_features = list(set(X_advanced.columns) & set(X_submission_enhanced.columns))
print(f"\nCommon features: {len(common_features)}")

# Ensure same feature order
X_submission_aligned = X_submission_enhanced[X_advanced.columns]
print(f"Aligned submission shape: {X_submission_aligned.shape}")

🔍 Feature Mismatch Debug:
X_advanced shape: (18239, 30)
X_submission_enhanced shape: (30000, 33)
X_advanced columns: 30
X_submission_enhanced columns: 33

X_advanced columns:
  1. carat
  2. cut_encoded
  3. color_encoded
  4. clarity_encoded
  5. depth
  6. table
  7. x
  8. y
  9. z
  10. volume
  11. surface_area
  12. carat_per_volume
  13. density
  14. aspect_xy
  15. aspect_xz
  16. cut_quality
  17. color_quality
  18. clarity_quality
  19. overall_quality
  20. log_carat
  21. log_volume
  22. carat_squared
  23. volume_squared
  24. overall_quality_squared
  25. carat_cubed
  26. volume_cubed
  27. overall_quality_cubed
  28. carat_quality
  29. volume_quality
  30. carat_bin

X_submission_enhanced columns:
  1. carat
  2. cut_encoded
  3. color_encoded
  4. clarity_encoded
  5. depth
  6. table
  7. x
  8. y
  9. z
  10. volume
  11. carat_per_volume
  12. surface_area
  13. aspect_ratio_xy
  14. aspect_ratio_xz
  15. depth_ratio
  16. table_ratio
  17. carat_cut_interaction

KeyError: "['aspect_xy', 'aspect_xz', 'cut_quality', 'color_quality', 'clarity_quality', 'overall_quality', 'log_carat', 'log_volume', 'overall_quality_squared', 'overall_quality_cubed', 'carat_quality', 'volume_quality', 'carat_bin'] not in index"

In [22]:
# === Fix Submission Features & Retry Conservative Model ===
print("🔧 Recreating submission features to match training data...")

# Start with the basic submission data
X_submission_fixed = test.copy()

# Apply same preprocessing as training data
print("Applying preprocessing...")

# Encode categorical variables
X_submission_fixed['cut_encoded'] = le_cut.transform(X_submission_fixed['cut'])
X_submission_fixed['color_encoded'] = le_color.transform(X_submission_fixed['color']) 
X_submission_fixed['clarity_encoded'] = le_clarity.transform(X_submission_fixed['clarity'])

# Feature engineering - same as training
print("Creating engineered features...")

# Basic engineered features
X_submission_fixed['volume'] = X_submission_fixed['x'] * X_submission_fixed['y'] * X_submission_fixed['z']
X_submission_fixed['surface_area'] = 2 * (X_submission_fixed['x']*X_submission_fixed['y'] + 
                                         X_submission_fixed['x']*X_submission_fixed['z'] + 
                                         X_submission_fixed['y']*X_submission_fixed['z'])
X_submission_fixed['carat_per_volume'] = X_submission_fixed['carat'] / (X_submission_fixed['volume'] + 1e-8)
X_submission_fixed['density'] = X_submission_fixed['carat'] / (X_submission_fixed['volume'] + 1e-8)
X_submission_fixed['aspect_xy'] = X_submission_fixed['x'] / (X_submission_fixed['y'] + 1e-8)
X_submission_fixed['aspect_xz'] = X_submission_fixed['x'] / (X_submission_fixed['z'] + 1e-8)

# Quality features
X_submission_fixed['cut_quality'] = X_submission_fixed['cut'].map(cut_quality)
X_submission_fixed['color_quality'] = X_submission_fixed['color'].map(color_quality)
X_submission_fixed['clarity_quality'] = X_submission_fixed['clarity'].map(clarity_quality)
X_submission_fixed['overall_quality'] = (X_submission_fixed['cut_quality'] + 
                                        X_submission_fixed['color_quality'] + 
                                        X_submission_fixed['clarity_quality']) / 3

# Log features
X_submission_fixed['log_carat'] = np.log1p(X_submission_fixed['carat'])
X_submission_fixed['log_volume'] = np.log1p(X_submission_fixed['volume'])

# Polynomial features
X_submission_fixed['carat_squared'] = X_submission_fixed['carat'] ** 2
X_submission_fixed['volume_squared'] = X_submission_fixed['volume'] ** 2
X_submission_fixed['overall_quality_squared'] = X_submission_fixed['overall_quality'] ** 2
X_submission_fixed['carat_cubed'] = X_submission_fixed['carat'] ** 3
X_submission_fixed['volume_cubed'] = X_submission_fixed['volume'] ** 3
X_submission_fixed['overall_quality_cubed'] = X_submission_fixed['overall_quality'] ** 3

# Interaction features
X_submission_fixed['carat_quality'] = X_submission_fixed['carat'] * X_submission_fixed['overall_quality']
X_submission_fixed['volume_quality'] = X_submission_fixed['volume'] * X_submission_fixed['overall_quality']

# Binning feature
X_submission_fixed['carat_bin'] = pd.cut(X_submission_fixed['carat'], bins=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])
le_bin = LabelEncoder()
X_submission_fixed['carat_bin'] = le_bin.fit_transform(X_submission_fixed['carat_bin'].astype(str))

# Select same features as training
feature_columns_fixed = X_advanced.columns.tolist()
X_submission_final = X_submission_fixed[feature_columns_fixed]

print(f"✅ Fixed submission shape: {X_submission_final.shape}")
print(f"Training shape: {X_advanced.shape}")
print(f"Features match: {list(X_submission_final.columns) == list(X_advanced.columns)}")

# Now generate conservative predictions with corrected features
print(f"\n🎲 Generating Conservative Predictions with Fixed Features...")
conservative_predictions = model_conservative.predict(X_submission_final, num_iteration=model_conservative.best_iteration)

# Create submission file
conservative_submission = pd.DataFrame({
    'id': range(len(conservative_predictions)),
    'price': conservative_predictions
})

# Save conservative model and predictions
cv_score = np.mean(cv_r2_scores)
conservative_filename = f"diamond_conservative_r2_{cv_score:.4f}.csv"
conservative_model_file = f"conservative_lgb_r2_{cv_score:.4f}.joblib"

conservative_submission.to_csv(conservative_filename, index=False)
joblib.dump(model_conservative, conservative_model_file)

print(f"\n✅ Conservative Model Completed:")
print(f"  Cross-validation R²: {cv_score:.4f} ± {np.std(cv_r2_scores):.4f}")
print(f"  Submission saved: {conservative_filename}")
print(f"  Model saved: {conservative_model_file}")
print(f"  Features: {len(feature_columns_fixed)}")

print(f"\n⚖️ Expected Performance:")
print(f"  Ultra Model: Validation {r2_ultra_lgb:.4f} → Kaggle 0.6188 (Gap: {r2_ultra_lgb - 0.6188:.4f})")
print(f"  Conservative: CV {cv_score:.4f} → Expected Kaggle ~{cv_score:.4f}")
print(f"  Predicted Kaggle improvement: {cv_score - 0.6188:.4f}")

print(f"\n🎯 Key Model Differences:")
print(f"  Ultra: 300 leaves, 15 depth, 0.01 lr, 8000 rounds")
print(f"  Conservative: 31 leaves, 6 depth, 0.05 lr, ~{int(np.mean([model.best_iteration for model in cv_models]))} rounds")
print(f"  Conservative adds: feature/bagging sampling + L1/L2 regularization")

🔧 Recreating submission features to match training data...
Applying preprocessing...
Creating engineered features...
✅ Fixed submission shape: (30000, 30)
Training shape: (18239, 30)
Features match: True

🎲 Generating Conservative Predictions with Fixed Features...

✅ Conservative Model Completed:
  Cross-validation R²: 0.6688 ± 0.0109
  Submission saved: diamond_conservative_r2_0.6688.csv
  Model saved: conservative_lgb_r2_0.6688.joblib
  Features: 30

⚖️ Expected Performance:
  Ultra Model: Validation 0.6603 → Kaggle 0.6188 (Gap: 0.0415)
  Conservative: CV 0.6688 → Expected Kaggle ~0.6688
  Predicted Kaggle improvement: 0.0500

🎯 Key Model Differences:
  Ultra: 300 leaves, 15 depth, 0.01 lr, 8000 rounds
  Conservative: 31 leaves, 6 depth, 0.05 lr, ~66 rounds
  Conservative adds: feature/bagging sampling + L1/L2 regularization
